In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from time import sleep
import pandas as pd
import re
from itertools import count
import time

In [11]:
def loginLVieclam24h(driver, url, username, password):
    driver.get(url)
    time.sleep(2)

    driver.find_element(By.NAME, "email").send_keys(username)
    driver.find_element(By.NAME, "password").send_keys(password)
    driver.find_element(By.NAME, "password").send_keys(Keys.RETURN)
    time.sleep(2)
    driver.find_element(By.XPATH, "//button[text()='Để sau']").click()
    return driver

In [12]:
options = Options()
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")

driver = webdriver.Chrome(options=options)

In [13]:
url = "https://ntd.vieclam24h.vn/tim-kiem-ung-vien-nhanh?per_page=10&salary_min=22220000&updated_at="
username = "hrai.aip391@gmail.com"
password = "minhvieclam24h"
loginLVieclam24h(driver, url, username, password)

<selenium.webdriver.chrome.webdriver.WebDriver (session="29608b32919397c5b061d3f3f03facd6")>

In [37]:
driver.find_element(By.XPATH, "//*[contains(@class, 'svicon-close') and contains(@class, 'text-[24px]')]").click()

In [38]:
elements = driver.find_elements(By.CLASS_NAME, "w-full.px-4.py-3.bg-white.border.rounded-sm.border-grey-light")

In [14]:
def extract_hrefs(driver):
    # Find all elements with the given class
    elements = driver.find_elements(By.CLASS_NAME, "w-full.px-4.py-3.bg-white.border.rounded-sm.border-grey-light")

    hrefs = []
    for element in elements:
        # Check if the element contains an anchor tag or is an anchor tag itself
        anchor = element.find_element(By.TAG_NAME, 'a')
        if anchor:
            href = anchor.get_attribute('href')
            if href:  # Ensure the href exists
                hrefs.append(href)
    return hrefs

In [15]:
def scroll_to_bottom(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")  # Get the height of the page

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # Scroll to the bottom
        time.sleep(1)  # Wait for the new content to load

        # Calculate new scroll height and compare it with the last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  # If heights are the same, we are at the bottom
            break
        last_height = new_height

In [16]:
df = pd.DataFrame(columns=['Link'])

In [21]:
for i in range(200):
    scroll_to_bottom(driver)
    hrefs = extract_hrefs(driver)
    df = pd.concat([df, pd.DataFrame(hrefs, columns=['Link'])], ignore_index=True)

    try:
        element = driver.find_element(By.XPATH, "//div[text()='Trang sau']")
        element.click()
        time.sleep(1)  # Wait for the next page to load
    except Exception as e:
        print(f"Error occurred: {e}")
        break  # Break the loop if the button is not found or clickable

KeyboardInterrupt: 

In [22]:
df = df.drop_duplicates()

In [23]:
len(df)

1990

In [24]:
df.to_csv("ALL.csv")